# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [7]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [8]:
# Read the WeatherPy_vacation.csv into a DataFrame
data_to_load = "../Vacation_Search/WeatherPy_vacation.csv"
vacation_df = pd.read_csv(data_to_load)

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,City Weather Description,Lat,Lng,Hotel Name
0,0,Hilo,US,83.28,few clouds,19.7297,-155.0900,Dolphin Bay Hotel
1,1,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,Grand Bel Air
2,4,Kapaa,US,80.92,clear sky,22.0752,-159.3190,Pono Kai Resort
3,9,Ambanja,MG,73.81,moderate rain,-13.6833,48.4500,Hôtel Salama Rose
4,12,Sao Filipe,CV,76.23,broken clouds,14.8961,-24.4956,Bela Vista


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [9]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "City Weather Description"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,City Weather Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [10]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Bowen"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Bowen"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Mareeba"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Kununurra"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Port Hedland"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [11]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], axis=0)

# Display sample data
itinerary_df.head()

,City_ID,City,Country,Max Temp,City Weather Description,Lat,Lng,Hotel Name
69,319,Bowen,AU,71.29,overcast clouds,-20.0167,148.2333,Whitsunday Sands Resort
74,354,Mareeba,AU,70.52,light rain,-17.0000,145.4333,The Gateway Hotel
121,542,Kununurra,AU,71.58,overcast clouds,-15.7667,128.7333,Kununurra Country Club Resort
146,629,Port Hedland,AU,78.85,broken clouds,-20.3167,118.5667,The Esplanade Hotel
69,319,Bowen,AU,71.29,overcast clouds,-20.0167,148.2333,Whitsunday Sands Resort


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [14]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()

# Display sample data
waypoints_df.head()

,Lat,Lng
69,-20.0167,148.2333
74,-17.0000,145.4333
121,-15.7667,128.7333
146,-20.3167,118.5667
69,-20.0167,148.2333


## Use GeoViews to create map that shows the four cities in the itinerary

In [19]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 100,
)

In [20]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [21]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [22]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'b64c80d1d3f8480ca4cb5d03be8101ce',
 'waypoints': '-20.0167,148.2333|-17.0,145.4333|-15.7667,128.7333|-20.3167,118.5667|-20.0167,148.2333'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [26]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [39]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response.['features'][0]['properties']['legs']

SyntaxError: invalid syntax (152500207.py, line 2)

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [ ]:
# Create and empty list to store the longitude of each step
# YOUR CODE HERE

# Create and empty list to store the latitude of step
# YOUR CODE HERE

# Loop through the legs coordinates to fetch the latitude and longitude for each step
# YOUR CODE HERE

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [ ]:
# Create an empty DataFrame to store the steps' coordinates
route_df = # YOUR CODE HERE

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [ ]:
# Configure the route path by using the GeoViews' Path function
route_path = # YOUR CODE HERE

In [ ]:
# Display a composed plot by using the route_map and route_path objects
# YOUR CODE HERE